In [27]:
! pip install pytesseract opencv-python pillow pdf2image


ERROR: Could not find a version that satisfies the requirement install (from versions: none)
ERROR: No matching distribution found for install


In [28]:
!pip install regex

In [ ]:
!sudo apt-get install tesseract-ocr


In [26]:
import cv2
from PIL import Image
import pytesseract


image = cv2.imread('/content/birth-certificate-babyg-quinn-fhir-sample-data_Page_1.png')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.resize(gray, None, fx=2, fy=2, interpolation=cv2.INTER_LINEAR)
gray = cv2.convertScaleAbs(gray, alpha=1.5, beta=0)
gray = cv2.GaussianBlur(gray, (5, 5), 0)
cv2.imwrite('preprocessed_image.jpg', gray)  #optional
text = pytesseract.image_to_string(gray)
print(text)



U.S. STANDARD CERTIFICATE OF LIVE BIRTH
LOCAL FILE NO. BIRTH NUMBER:

1. CHILD'S NAME (First, Middle, Last, Suffix } 2. TIME OF BIRTH [3.5 4. DATE OF BIRTH (Mo/Day/Yr)}
Baby G Quinn 1300 (24 hr) 02/12/2019

5. FACILITY NAME (If not institution, give street and number) 6. CITY, TOVVWN, OR LOCATION OF BIRTH

South Hospital (21 North Avenue) Salt Lake City, UT, 84116

M O T H E R 8a. MOTHER'S CURRENT LEGAL NAME (First, Middle, Last, Suffix) 8b. DATE OF BIRTH (Ma/Day/yr)
MOTHER Jada Ann Quinn 01/15/1985

8c. MOTHER'S NAME PRIOR TO FIRST MARRIAGE (First, Middle, Last, Suffix } 8d. BIRTHPLACE (State, Terntory, or Foreign Country)
Jada Ann King UT

 
 
  
  
    
    

 
    

9a. RESIDENCE OF MOTHER-STATE 9b. COUNTY 9c. CITY, TOWN, OR LOCATION
UT Salt Lake Salt Lake City
Od. STREET AND NUMBER ge. APT. NO of. ZIP CODE

1875 West Morton Avenue 84116

LIMITS?

 

F A T H E R 10a. FATHER'S CURRENT LEGAL NAME (First, Middle, Last, Suffix) 10b. DATE OF BIRTH (Mo/Day/fyr) 10c. BIRTHPLACE (State, Te

In [22]:
#config 1


custom_config = r'--oem 3 --psm 6'
text = pytesseract.image_to_string(gray, config=custom_config)

print(text)


Software Design Documentation

Product Name ve

Date Updated i

Written By os
Introduction
The purpose of this document 's to odrov'de a comprenensive overv ew of tne software desicn for the
XYZ application. This includes the system overview, design considerations, speci*ications, detailed
design, mplementat on plan, testing alan, and maintenance olan.
System Overview
The XYZ application 's a wed-Dased platform designed to streamline the process of project
management. It allows users to create projects, assign tasks, track progress, and communicate with
team members. Tne application will de ouilt using a comdination of HTML, CSS, and JavaScript for the
‘Frontend and Node.js for the Dacxend.
Design Considerations

- User-frendly interface

¢ Scalability to accommodate 2 large 1umBer of users and projects

« Security measures to protect user data

« Compatibility witn different browsers and devices

« Integration with external APIs for add'tional functionalty
Design Specifications

Requi

In [30]:
import cv2
import numpy as np
import pytesseract
from pdf2image import convert_from_path
import re
import os

# Ensure you have Tesseract installed and the path set correctly
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'  # Update this path as needed

def convert_pdf_to_images(pdf_path):
    return convert_from_path(pdf_path)

def preprocess_image(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply adaptive thresholding
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

    # Denoise
    denoised = cv2.fastNlMeansDenoising(thresh, None, 10, 7, 21)

    # Enhance contrast
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced = clahe.apply(denoised)

    return enhanced

def extract_text(image, lang='eng+hin+ben'):
    config = r'--oem 3 --psm 6'
    return pytesseract.image_to_string(image, lang=lang, config=config)

def detect_document_type(text):
    text = text.lower()
    if any(word in text for word in ['aadhaar', 'आधार', 'অধার']):
        return "Aadhaar Card"
    elif any(word in text for word in ['permanent account number', 'pan', 'पैन', 'স্থায়ী অ্যাকাউন্ট নম্বর']):
        return "PAN Card"
    elif any(word in text for word in ['birth certificate', 'जन्म प्रमाण पत्र', 'জন্ম শংসাপত্র']):
        return "Birth Certificate"
    elif any(word in text for word in ['mark sheet', 'अंक तालिका', 'মার্কশিট']):
        if any(word in text for word in ['10th', 'x', 'दसवीं', 'দশম']):
            return "Class 10 Mark Sheet"
        elif any(word in text for word in ['12th', 'xii', 'बारहवीं', 'দ্বাদশ']):
            return "Class 12 Mark Sheet"
    return "Unknown Document"

def extract_aadhaar_info(text):
    aadhaar_pattern = r'\d{4}\s\d{4}\s\d{4}'
    match = re.search(aadhaar_pattern, text)
    return {'Aadhaar Number': match.group() if match else "Not found"}

def extract_pan_info(text):
    pan_pattern = r'[A-Z]{5}[0-9]{4}[A-Z]'
    match = re.search(pan_pattern, text)
    return {'PAN Number': match.group() if match else "Not found"}

def extract_birth_certificate_info(text):
    date_pattern = r'\d{2}[/-]\d{2}[/-]\d{4}'
    name_pattern = r'Name[:\s]+([A-Za-z\s]+)'

    date_match = re.search(date_pattern, text)
    name_match = re.search(name_pattern, text)

    return {
        'Date of Birth': date_match.group() if date_match else "Not found",
        'Name': name_match.group(1) if name_match else "Not found"
    }

def extract_marksheet_info(text):
    total_marks_pattern = r'Total[:\s]+(\d+)'
    percentage_pattern = r'Percentage[:\s]+([\d.]+)%?'

    total_marks_match = re.search(total_marks_pattern, text)
    percentage_match = re.search(percentage_pattern, text)

    return {
        'Total Marks': total_marks_match.group(1) if total_marks_match else "Not found",
        'Percentage': percentage_match.group(1) if percentage_match else "Not found"
    }

def process_document(file_path):
    if file_path.lower().endswith('.pdf'):
        images = convert_pdf_to_images(file_path)
    else:
        images = [cv2.imread(file_path)]

    all_text = ""
    for img in images:
        preprocessed = preprocess_image(np.array(img))
        text = extract_text(preprocessed)
        all_text += text + "\n"

    doc_type = detect_document_type(all_text)

    if doc_type == "Aadhaar Card":
        info = extract_aadhaar_info(all_text)
    elif doc_type == "PAN Card":
        info = extract_pan_info(all_text)
    elif doc_type == "Birth Certificate":
        info = extract_birth_certificate_info(all_text)
    elif "Mark Sheet" in doc_type:
        info = extract_marksheet_info(all_text)
    else:
        info = {"Info": "No specific information extracted"}

    return doc_type, info, all_text

def main():
    file_path = input("Enter the path to your document (image or PDF): ")

    if not os.path.exists(file_path):
        print("File not found. Please check the path and try again.")
        return

    doc_type, info, full_text = process_document(file_path)

    print(f"\nDetected Document Type: {doc_type}")
    print("\nExtracted Information:")
    for key, value in info.items():
        print(f"{key}: {value}")

    print("\nFull Extracted Text:")
    print(full_text)

if __name__ == "__main__":
    main()

Enter the path to your document (image or PDF): /content/birth-certificate-babyg-quinn-fhir-sample-data_Page_1.png

Detected Document Type: PAN Card

Extracted Information:
PAN Number: Not found

Full Extracted Text:
U.S. STANDARD CERTIFICATE OF LIVE BIRTH
LOCAL FILE NO. BIRTH NUMBER:
CH LL DI SHLOSNAME Gist, wicale, Last. Sutfix 2. TIME OF BIRTH ]3.SEX — [4 DATE OF BIRTH (Mo/Day/Yr}
Baby G Quinn 1300 (24 hr) F 02/12/2019
5. FACILITY NAME (If not institution, give street and number) 6. CITY, TOWN, OR LOCATION OF BIRTH 7. COUNTY OF BIRTH
South Hospital (21 North Avenue) Salt Lake City, UT, 84116 Salt Lake
M OTHE R|® Movers CRRENT LEGAL NAME (Fist Middle, Last, Sutfc} 8b. DATE OF BIRTH (MolDayivn)
Jada Ann Quinn 01/15/1985
8¢. MOTHER'S NAME PRIOR TO FIRST MARRIAGE (First, Middle, Last, Suffix) 8d. BIRTHPLACE (State, Termitory, or Foreign Country)
Jada Ann King UT
9a. RESIDENCE OF MOTHER-STATE | 9b. COUNTY 9c. CITY, TOWN, OR LOCATION
UT Salt Lake Salt Lake City
SSTINSTOE CITY
9d. STREET A